# Using phrasal segmentation results to analyze phrases over time

In [1]:
import pandas as pd
import re
from glob import glob
import numpy as np
import json
import altair as alt
import csv

## Initial loading + processing of dataframes and dictionaries

In [2]:
# AutoPhrase results dataframe
fp_phrases = '../results/dblp-v10-grouped/phrases.csv'
phrases = pd.read_csv(fp_phrases, index_col=0)
phrases.head()

,Phrase Quality,Phrase,Year,Num Words
0,0.9915,operations research,1950-1959,2
1,0.6505,operations research society of america,1950-1959,5
2,0.5735,high speed,1950-1959,2
3,0.5255,operations research society,1950-1959,3
4,0.9810,tunnel diode,1960-1964,2


In [3]:
# Processed phrasal segmentation results dataframe
infolder = '../results/dblp-v10-grouped'
subfolders = glob(infolder + '/*.csv')
subfolders = list(filter(lambda x: 'segmented' in x, subfolders))
seg = pd.DataFrame(columns=['Phrases', 'Year', 'Num Phrases'])
for fp in subfolders:
    df = pd.read_csv(fp, index_col=0)
    df = df.dropna()
    df['Num Phrases'] = df.apply(lambda x: len(x['Phrases'].split(',')), axis=1)
    #df = df.drop('Phrases', axis=1)
    seg = seg.append(df, ignore_index=True)
seg = seg.dropna()
seg.head()

,Phrases,Year,Num Phrases
0,"paper,wheatstone,bridge,tangent,triangle,trian...",1950-1959,14
1,"numerical integration,differential equations,o...",1950-1959,6
2,fur,1950-1959,1
3,"computing,computing,amplifier,high,amplifiers,...",1950-1959,8
4,"operations research,journal,operations researc...",1950-1959,5


In [4]:
# Creates dictionary for counts of phrases for each year range
counts = {}
for yr in seg['Year'].unique():
    counts[yr] = {}
    
def add_counts(x):
    phrases = x['Phrases'].split(',')
    year = x['Year']
    for phrase in phrases:
        if phrase not in counts[year]:
            counts[year][phrase] = 0
        counts[year][phrase] += 1

_ = seg.apply(add_counts, axis=1)

# Sorts the counts in descending order
for key, val in counts.items():
    counts[key] = dict(sorted(val.items(), key=lambda item: item[1], reverse=True))

In [5]:
# Only contains the multi-word phrases and counts
multi_counts = {}
for year_range, phrase_counts in counts.items():
    multi_counts[year_range] = {key: val for key, val in phrase_counts.items() if len(key.split()) > 1}

In [6]:
# Creates counts dictionary but with percent as values, rather than raw frequency
counts_per = {}
for year_range, phrase_counts in counts.items():
    total_count_yr = sum(phrase_counts.values())
    prop_counts = {}
    for key, val in phrase_counts.items():
        prop_counts[key] = (val / total_count_yr)
    counts_per[year_range] = prop_counts

In [ ]:
# Looking at most popular multi word phrases in each year range

In [41]:
BAD_PHRASES_POP = set(['per cent', 'es wird', 'neural networks', 'proposed method', 'authors present',
                        'proposed algorithm', 'proposed approach', 'ad hoc', 'a case study',
                        'proposed scheme', 'recent years', 'numerical results', 'an adaptive',
                        'sufficient conditions', 'special case', 'john wiley  sons', 'real time', 'wide range', 'computer science',
                        'technical note'])

In [56]:
most_pop = {}
for key, val in multi_counts.items():
    top_10 = []
    val_phrases = list(val.items())
    for tup in val_phrases:
        if tup[0] not in BAD_PHRASES_POP:
            top_10.append(tup)
        if len(top_10) == 10:
            break
    most_pop[key] = top_10
most_pop

{'1950-1959': [('operations research', 82),
  ('gaussian noise', 16),
  ('differential equation', 12),
  ('dynamic programming', 8),
  ('standard model', 8),
  ('signal-to-noise ratio', 7),
  ('differential equations', 6),
  ('machine language', 6),
  ('linear programming', 6),
  ('formal logic', 6)],
 '1960-1964': [('pattern recognition', 27),
  ('regular expressions', 22),
  ('differential equations', 21),
  ('linear programming', 19),
  ('sequential circuits', 15),
  ('numerical integration', 11),
  ('dynamic programming', 11),
  ('differential equation', 11),
  ('gaussian noise', 10),
  ('transfer function', 10)],
 '1965-1969': [('sequential machines', 85),
  ('pattern recognition', 75),
  ('linear programming', 71),
  ('analog computer', 58),
  ('sequential machine', 54),
  ('dynamic programming', 49),
  ('switching functions', 47),
  ('differential equations', 45),
  ('threshold functions', 45),
  ('flow table', 42)],
 '1970-1974': [('pattern recognition', 165),
  ('linear progra

In [57]:
data = {}
for key, val in most_pop.items():
    val_proc = []
    for item in val:
        val_proc.append(item[0] + ' (' + str(item[1]) + ')')
    data[key] = val_proc

In [58]:
pd.DataFrame.from_dict(data).to_csv('../results/top10.csv', index=False)

## Start of phrases over time EDA + figures

In [7]:
# TODO:
# Ridgeline plot where y-axis is a Phrase, x-axis is the Year Range, height showing the frequency or percentage of each phrase

In [8]:
# Add more phrases to list by looking at the dictionaries
interesting_phrases = ['cloud computing', 'machine learning', 'neural network', 'data mining', 'social networks',
                      'image processing', 'computer vision', 'user experience', 'genetic algorithms', 'computer science',
                      'virtual reality']

In [9]:
# Looking at count (raw frequency) of each phrase across year ranges
data = pd.DataFrame(columns=['Year Range', 'Count', 'Phrase'])
phrases_to_graph = ['machine learning', 'computer vision', 'neural network']

for phrase in phrases_to_graph:
    for key, val in counts.items():
        count = val.get(phrase)
        data.loc[len(data)] = [key, count, phrase]

In [10]:
# Issue with looking at raw frequency is that it will depend on how many papers are in each year range
# There are more papers over time, so the frequency will naturally increase for almost all phrases
alt.Chart(data).mark_line().encode(x='Year Range', y='Count', color='Phrase')

alt.Chart(...)

In [15]:
# Looking at percentage of each phrase across year ranges
phrases_to_graph = ['machine learning', 'computer vision', 'neural network', 'virtual reality', 'computer science']

data = pd.DataFrame(columns=['Year Range', 'Percent', 'Phrase'])
for phrase in phrases_to_graph:
    for key, val in counts_per.items():
        count = val.get(phrase)
        data.loc[len(data)] = [key, count, phrase]
data = data.fillna(0)

In [16]:
alt.Chart(data).mark_line().encode(x='Year Range',
                                   y=alt.Y('Percent', axis=alt.Axis(format='%')),
                                   color='Phrase'
                                   ).properties(title='Phrase Percent Prominence over Year Ranges')

alt.Chart(...)

In [14]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Year Range'], empty='none')

line = alt.Chart(data).mark_line(interpolate='basis').encode(
    x='Year Range',
    y=alt.Y('Percent', axis=alt.Axis(format='%')),
    color='Phrase')

selectors = alt.Chart(data).mark_point().encode(
    x='Year Range',
    opacity=alt.value(0)).add_selection(nearest)

points = line.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))
    
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Percent', alt.value(' ')))
    
rules = alt.Chart(data).mark_rule(color='gray').encode(
    x='Year Range').transform_filter(nearest)

alt.layer(line, selectors, points, rules, text).properties(width=600,
                                                           height=300,
                                                           title='Phrase Percent Prominence over Year Ranges (interactive)')

alt.LayerChart(...)

In [15]:
alt.Chart(data).mark_area(opacity=0.5).encode(x='Year Range', 
                                              y=alt.Y('Percent', stack=None), 
                                              color='Phrase',
                                              tooltip=['Phrase', 'Year Range', 'Percent']
                                              ).properties(title='Phrase Percent Prominence over Year Ranges').interactive()

alt.Chart(...)

# Gephi data pre-processing

In [16]:
# Gephi segmentation csvs only contain high-quality, multi-word phrases (no duplicates per paper)
infolder = '../results/gephi'
subfolders = glob(infolder + '/*.csv')
#subfolders = list(filter(lambda x: 'segmented' in x, subfolders))
data = pd.DataFrame(columns=['Phrases', 'Year Range'])
for fp in subfolders:
    df = pd.read_csv(fp, index_col=0)
    df = df.dropna()
    #df['Num Phrases'] = df.apply(lambda x: len(x['Phrases'].split(',')), axis=1)
    #df = df.drop('Phrases', axis=1)
    data = data.append(df, ignore_index=True)
data = data.dropna()
data['Phrases'] = data['Phrases'].map(lambda x: x.split(','))

In [17]:
# Can use to remove papers with only a single phrase
data = data[data.apply(lambda x: len(x['Phrases']) > 1, axis=1)]

In [18]:
# Use to see the distribution of phrase counts per paper
# data['Phrases'].map(len).value_counts()

In [19]:
labels = pd.DataFrame(columns=['ID', 'Label', 'Count'])
labels

,ID,Label,Count


In [20]:
bad_phrases = set(['fur', 'e', 'ma', 'fr', 'as', 'first', 'most', 'so', 'if', 'ii', 'i', 'k',
                   'm', 'd', 'far', 'b', 'co', 't', 's', 'h', 'et'])
to_add = ['proposed method', 'proposed approach', 'recent years']

In [21]:
label_counts = {}
def get_label_counts(x):
    for phrase in x:
        # if phrase in bad_phrases: continue
        if phrase not in label_counts:
            label_counts[phrase] = 0
        label_counts[phrase] += 1

_ = data.apply(lambda x: get_label_counts(x['Phrases']), axis=1)
label_counts = dict(sorted(label_counts.items(), key=lambda item: item[1], reverse=True))

In [22]:
labels = pd.DataFrame.from_dict(label_counts, orient='index', columns=['Count']).reset_index().rename(columns={'index': 'ID'})
labels['Label'] = labels['ID']
labels = labels[['ID', 'Label', 'Count']]
labels

,ID,Label,Count
0,proposed method,proposed method,90379
1,proposed approach,proposed approach,44371
2,proposed algorithm,proposed algorithm,40898
3,a case study,a case study,32366
4,recent years,recent years,28936
...,...,...,...
84292,catastrophic events,catastrophic events,1
84293,minority over sampling,minority over sampling,1
84294,semi dense,semi dense,1
84295,k user mimo,k user mimo,1


In [23]:
labels[labels['Count'] > 300].to_csv('../results/NodeData.csv')

In [24]:
#labels.to_csv('../results/NodeData.csv')

In [25]:
data

,Phrases,Year Range
0,"[positive definite, numerical integration, dif...",1950-1959
2,"[machine language, future systems]",1950-1959
3,"[operations research, linear programming]",1950-1959
4,"[power series, round off error]",1950-1959
7,"[operations research, linear programming]",1950-1959
...,...,...
2471993,"[level set, current state, remains unclear, vo...",2015-2017
2471994,"[extended kalman filter ekf, received signal s...",2015-2017
2471996,"[infinite horizon, dynamic pricing, discount r...",2015-2017
2471997,"[per minute, operating conditions, m 1, power ...",2015-2017


In [26]:
edge_dict = {}

def get_edges(phrase_lst):

    for phrase in phrase_lst:
        for inner_phrase in phrase_lst:
            # Prevents any self-comparisons
            if phrase == inner_phrase: continue

            # Stops any comparisons of existing phrase A - phrase B comparisons
            # We don't need to add the phrase B - phrase A data to the dictionary
            if inner_phrase in edge_dict and phrase in edge_dict[inner_phrase]:
                continue

            if phrase not in edge_dict:
                edge_dict[phrase] = {}

            if inner_phrase not in edge_dict[phrase]:
                edge_dict[phrase][inner_phrase] = 0
            edge_dict[phrase][inner_phrase] += 1

In [27]:
# test = pd.DataFrame(['a,b,c', 'b,c,a', 'c,b', 'a,b,c'], columns=['Phrases'])
# test['Phrases'] = test.apply(lambda x: x['Phrases'].split(','), axis=1)
# test

In [28]:
# _ = test.apply(lambda x: get_edges(x['Phrases']), axis=1)
# edge_dict

In [29]:
_ = data.apply(lambda x: get_edges(x['Phrases']), axis=1)

In [30]:
#edge_dict

In [31]:
threshold = 300
edge_filtered = {}
for phrase, phrase_counts in edge_dict.items():
    edge_filtered[phrase] = {}
    for inner_phrase, count in phrase_counts.items():
        if count < threshold:
            continue
        edge_filtered[phrase][inner_phrase] = count

In [32]:
edge_filtered

{'positive definite': {},
 'numerical integration': {},
 'machine language': {},
 'operations research': {},
 'power series': {},
 'numerical method': {'numerical results': 324},
 'linear function': {},
 'network theory': {},
 'inverse problem': {},
 'steady state': {},
 'negative feedback': {},
 'angular frequency': {},
 'inventory control': {},
 'chemical process': {},
 'north american': {},
 'standard language': {},
 'san diego': {},
 'error detection': {},
 'world war': {},
 'standard model': {},
 'pulse repetition frequency': {},
 'ordinary differential equations': {},
 'differential equations': {'differential equation': 326},
 'quadratic equation': {},
 'electric field': {},
 'space charge': {},
 'reflection coefficient': {},
 'statistical model': {},
 'jet propulsion laboratory': {},
 'conditional probability': {},
 'false alarm': {},
 'likelihood ratio': {},
 'impulse response': {},
 'closed form': {},
 'level crossings': {},
 'statistical theory': {},
 'dependent variable': {}

In [33]:
import csv

In [34]:
# Testing with only including nodes in the EdgeData in NodeData

In [35]:
edge_phrases = set()

In [36]:
for phrase, phrase_counts in edge_filtered.items():
    for inner_phrase, count in phrase_counts.items():
        edge_phrases.add(inner_phrase)
        edge_phrases.add(phrase)

In [37]:
edge_phrases

{'a case study',
 'access control',
 'access network',
 'access networks',
 'activation function',
 'activation functions',
 'ad hoc',
 'adaptive control',
 'admission control',
 'amino acid',
 'amino acids',
 'an ad hoc network',
 'an adaptive',
 'an improved',
 'an open source',
 'an overview',
 'analytical model',
 'analytical results',
 'anomaly detection',
 'approximation algorithm',
 'approximation algorithms',
 'approximation ratio',
 'artificial intelligence',
 'artificial neural network',
 'artificial neural network ann',
 'artificial neural networks',
 'association rule mining',
 'association rules',
 'asymptotic stability',
 'augmented reality',
 'augmented reality ar',
 'automatic speech recognition',
 'automatic speech recognition asr',
 'back propagation',
 'base station',
 'base stations',
 'based access control',
 'based algorithm',
 'based approach',
 'based method',
 'based methods',
 'benchmark datasets',
 'benchmark functions',
 'benchmark problems',
 'ber performan

In [38]:
label_counts = {}
def get_label_counts(x):
    for phrase in x:
        # Skips to next phrase if it is not in the EdgeData
        if phrase not in edge_phrases:
            continue

        if phrase not in label_counts:
            label_counts[phrase] = 0
        label_counts[phrase] += 1

_ = data.apply(lambda x: get_label_counts(x['Phrases']), axis=1)
label_counts = dict(sorted(label_counts.items(), key=lambda item: item[1], reverse=True))

In [39]:
labels = pd.DataFrame.from_dict(label_counts, orient='index', columns=['Count']).reset_index().rename(columns={'index': 'ID'})
labels['Label'] = labels['ID']
labels = labels[['ID', 'Label', 'Count']]
labels

,ID,Label,Count
0,proposed method,proposed method,90379
1,proposed approach,proposed approach,44371
2,proposed algorithm,proposed algorithm,40898
3,a case study,a case study,32366
4,recent years,recent years,28936
...,...,...,...
713,an ad hoc network,an ad hoc network,487
714,interference alignment ia,interference alignment ia,462
715,obtainable from cpc program library queens,obtainable from cpc program library queens,441
716,based access control,based access control,433
